# CV Project2

## Image Classification Project

```
Category - Cat and Dog
Tech used - VGGG-16, Roboflow, Keras
Techniques - Transfer Learning

```


# Implementation of Transfer Learning in VGG-16 model

In [ ]:
!nvidia-smi

: 

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#ROOT_PATH = '/content/drive/MyDrive/005_BOKTIAR_AHMED_BAPPY/My_classes/FSDS-Bootcamp/cat-dog-project'
#my_root_path = 'https://drive.google.com/drive/u/0/folders/17SsOmM6NQ9tkDIkE4fsHlt5qc-O74B3c'

In [ ]:
my_root_path = '/content/drive/MyDrive'

In [ ]:
!pwd

In [ ]:
os.chdir(my_root_path)
os.getcwd()

In [ ]:
!ls

# Importing the VGG-16 model

We are importing the pre-trained VGG16 model along with its weights. For this, we will keep `layer.trainable = False`.

- As the pre-trained model already learned about some important features like colors, edges, textures etc., we don't ned to train our model from scratch.

- So we will just make some changes in the VGG-16 model to utilyze this existing knowledge to solve our `Image classification` problem.

- It redces the training time and resources. It also doesn't need much data as the model is already trained on millions of data.



## VGG-16 Architecture:
```
2C,1P, 2C,1P, 3C, 1P, 3C, 1P, 3C, 1P, 3D layers

2   +   2   +  3   +  3   + 3   +      3   = 16 layers

```

where,

C = Convolution layer

P = Max Polling layer

D = Dense layer


CHANGE : We will not take last 3 Dense layers, i.e. the `top layers`. Instead we will add the top layers by ourselves.


In [ ]:
IMAGE_SIZE = [224, 224]

train_path = 'CatDog/train'
valid_path = 'CatDog/valid'
test_path = 'CatDog/test'

In [ ]:
vgg16_model = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [ ]:
IMAGE_SIZE + [3]

In [ ]:
# We do not change existing weights during training.
for layer in vgg16_model.layers:
  layer.trainable = False

In [ ]:
# To check the above
for layer in vgg16_model.layers:
  print(layer.name, layer.trainable)

In [ ]:
vgg16_model.summary()

# Importing Images of cats and dogs (from the drive)

In [ ]:
folders = glob('CatDog/train/*')
folders

In [ ]:
# To see categories of data
len(folders)

# Adding some changes to our pre-trained VGG-16 model

In [ ]:
model = Sequential()

model.add(vgg16_model)
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.summary()

*NOTE:* If you update the model after compiling the model, it becomes `NoneType`, so don't update the model after compiling like:

```
model = model.compile()
```

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
type(model)

# Preprocess the training images

In [ ]:
# Using the ImageDataGenerator to import the images from the dataset

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

# Creating Train and test dataset from datagen

In [ ]:
train_set = train_datagen.flow_from_directory(
    'CatDog/train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
test_set  = test_datagen.flow_from_directory(
    'CatDog/valid',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
m_his = model.fit(train_set,
                  validation_data = test_set,
                  epochs = 10,
                  steps_per_epoch = len(train_set),
                  validation_steps= len(test_set))

# Visualize the performance

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plotting the loss
plt.plot(m_his.history['loss'], label = 'train_loss')
plt.plot(m_his.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()
plt.savefig('loss_graph')

In [ ]:
# Plotting the accuracy
plt.plot(m_his.history['accuracy'], label = 'train_accuracy')
plt.plot(m_his.history['val_accuracy'], label = 'val_accuracy')
plt.legend()
plt.show()
plt.savefig('accuracy_graph')

# Saving the VGG-16 MODIFIED model

In [ ]:
#model.save('vgg19_modified.h5')
#model.model.save('vgg19_modified_model.h5')      # model is History class's object, it doesn't have save() but have 'model' property and it has save()

model.save('vgg16_modified_model.keras')

# Prediction / Classification

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

In [ ]:
import numpy as np

y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred

# Displaying Image classification of cats and dogs

In [ ]:
# Loading the saved model
mod = load_model('vgg16_modified_model.keras')

In [ ]:
type(mod)

In [ ]:
test_img = load_img('CatDog/test/dog/7_jpg.rf.328af46a3c47760288da55cb4578c658.jpg')

In [ ]:
type(test_img)

In [ ]:
# Converting this image to array

x = image.img_to_array(test_img)
x

In [ ]:
z = plt.imread('CatDog/test/dog/7_jpg.rf.328af46a3c47760288da55cb4578c658.jpg')
plt.imshow(z)

In [ ]:
z.shape

In [ ]:
x.shape

## Preprocess test input image

In [ ]:
x = x/255
x

In [ ]:
from keras.applications.vgg16 import preprocess_input

In [ ]:
import numpy as np
x = np.expand_dims(x, axis = 0)
img_data = preprocess_input(x)
img_data.shape

In [ ]:
test_pred = mod.predict(img_data)

In [ ]:
test_pred       # [cat, dog]

In [ ]:
result = np.argmax(test_pred, axis = 1)
result

In [ ]:
result[0]

In [ ]:
if result[0] == 1:
  test_pred = 'dog'
  print(test_pred)

else:
  test_pred = 'cat'
  print(test_pred)

# NOTE: Correct Classification of the given input dog image.